# StyleGAN2-ADA: Style Conversion

참고
github의 [stylegan2-ada-pytorch](https://github.com/NVlabs/stylegan2-ada-pytorch) 를 참고할 것



## 1단계 : 이미 학습된 모델을 가져오고, 초기 설정하기

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## 라이브러리 갖고 오기

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from PIL import Image

import numpy as np

import copy
import pickle
import os

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

## stylegan2-ada를 git의 저장소에서 복사해오기

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch

In [ ]:
if not os.path.isdir("pretrained"):
  !mkdir pretrained
  %cd pretrained
  !wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pickle
  %cd ../

In [ ]:
with open('pretrained/ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].cuda()

## 2단계 - 개 그림 샘플 만들기

In [ ]:
z = torch.randn([1, G.z_dim]).cuda()
c = None
img = G(z, c)

In [ ]:
img.size()

In [ ]:
torch.min(img)

In [ ]:
img.squeeze(0).size()

In [ ]:
plt.imshow((img[0]*127.5+128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.uint8))
plt.axis('off')
plt.show()

## 2 - submodule  g_mapping, g_synthesis
G model을 submodel로 나누어 사용할 수 있음


In [ ]:
g_mapping = G.mapping
g_synthesis = G.synthesis

In [ ]:
g_mapping

In [ ]:
g_synthesis

latent vector z를 random 가우시안 확률 분포에서 획득

In [ ]:
z = torch.randn(([1, G.z_dim])).cuda()
z.size()

z를 중간의 latent space w로 변환(disentangled space)

In [ ]:
w = g_mapping(z, None)
w.size()

w를 synthesis network로 보내서 그림을 만들게 함

In [ ]:
img = g_synthesis(w)
img = img.cpu()

In [ ]:
plt.imshow((img[0]*127.5 + 128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.unit8))
plt.axis('off')
plt.show()

## 3 - 그림 편집

그림을 다른 특성을 추가하여 부드럽게 바꾸기

2개의 random vector z와 중간 latent space w값 2개 얻기

In [ ]:
z1 = torch.randn([1, G.z_dim]).cuda()
z2 = torch.randn([1, G.z_dim]).cuda()

w1 = g_mapping(z1, None)
w2 = g_mapping(z2, None)

3- 2개의 그림 생성

In [ ]:
img1 = g_synthesis(w1)
img2 = g_synthesis(w2)

img1 = img1.squeeze().permute(1,2,0)
img2 = img2.squeeze().permute(1,2,0)

In [ ]:
imgs = torch.cat([img1, img2], axis=1).cpu()
imgs.size()

In [ ]:
imgs = (imgs * 127.5 + 128).clamp(0, 255).to(torch.uint8)

In [ ]:
plt.imshow(imgs)
plt.axis('off')
plt.show()

In [ ]:
img_group = []
lin = np.linspace(0, 1, 10)
print(*lin)


In [ ]:
with torch.no_grad():
  for i in lin:
    w = ((1-i) * w1) + ( i * w2) # w1에서 w2로 점차 변환
    result = g_synthesis(w)
    result = result.squeeze()
    img_group.append(result)

In [ ]:
img_group = torch.cat(img_group).cpu()
img_group.size()

점차 바뀌는 이미지를 하나로 모아서 한 번에 보기

In [ ]:
grid_img = torchvision.utils.make_grid(img_group, nrow=5)
grid_img.size()

In [ ]:
grid_img = (grid_img * 127.5 + 128).clamp(0, 255).to(torch.uint8)

In [ ]:
plt.figure(figsize=(16, 6))
plt.imshow(grid_img.permute(1,2,0))
plt.axis('off')
plt.show()

## 5. Style Conversion

그림 생성에서 보다 더 다양한 컨트롤 방법을 더하기 위해서 synthesis network는 그림의 꼴, 모습을 바꾸는데 다양한 형태를 제공함

Coarse - (4x4 - 8x8) - 자세, 머리카락, 얼굴 윤곽선
Middle - (16x16 - 32x32) - 보다 자세한 얼굴 형태, 머리카락, 눈 뜨고, 감은 정도
Fine - (64x64 - 1024x1024) - 다양한 색깔들(눈, 머리카락, 피부), 그리고 세부적인 특징들

In [ ]:
plt.imshow(imgs)
plt.axis('off')
plt.show()

In [ ]:
w_coarse = torch.cat((w2[:,:2], w1[:,:2]), dim=1)
img = g_synthesis(w_coarse)
img = img.cpu()

plt.imshow((img[0]*127.5 + 128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.uint8))
plt.axis('off')
plt.show()

In [ ]:
w_middle = torch.cat((w1[:,:2], w2[:,2:4], w1[:,4:]), dim=1)
img = g_synthesis(w_middle)
img = img.cpu()

plt.imshow((img[0]*127.5 + 128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.uint8))
plt.axis('off')
plt.show()

In [ ]:
w_fine = torch.cat((w2[:,:4], w1[:,4:]), dim=1)
img = g_synthesis(w_fine)
img = img.cpu()

plt.imshow((img[0]*127.5 + 128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.uint8))
plt.axis('off')
plt.show()